In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df1 = pd.read_parquet("hf://datasets/Sahi19/IndianLawComplete/formatted_dataset.parquet")

In [3]:
def split_text(row):
    match = re.search(r"\[INST\](.*?)\[/INST\](.*)", row)
    if match:
        question = match.group(1).strip()
        answer = match.group(2).strip()
        return question, answer
    return None, None

In [4]:
# Apply the function to the dataframe
df1[['question', 'answer']] = df1['text'].apply(lambda x: pd.Series(split_text(x)))

# Dropping the original text column 
df1.drop(columns=['text'], inplace=True)

# Save the processed dataset
df1.to_csv('processed_dataset.csv', index=False)

df1.head()

question  \
0  What is the main purpose of the Indian Penal C...   
1  Can you explain Section 2 regarding the punish...   
2  What does Section 3 say about punishment for o...   
3  How does Section 4 deal with extra-territorial...   
4  Is there any provision in the Indian Penal Cod...   

                                              answer  
0  The main purpose of the Indian Penal Code is o...  
1  Section 2 explains that every person shall be ...  
2  Section 3 states that the provisions of this C...  
3  Section 4 explains that any person liable, by ...  
4  Yes, Section 5 of Chapter 1 specifies that not...

In [5]:
splits = {'train': 'train.csv', 'validation': 'validation.csv', 'test': 'test.csv'}
df2 = pd.read_csv("hf://datasets/jizzu/llama2_indian_law_v2/" + splits["train"])

In [6]:
def split_question_answer(row):
    if "###Human:" in row and "###Assistant:" in row:
        question = row.split("###Human:")[1].split("###Assistant:")[0].strip()
        answer = row.split("###Assistant:")[1].strip()
        return question, answer
    return None, None

df2[['question', 'answer']] = df2['text'].apply(split_question_answer).apply(pd.Series)

print(df2.head())
print(df2.isnull().sum())

df2.to_csv("corrected_df2.csv", index=False)

                                                text  \
0  ###Human:\nWhat is the difference between a pe...   
1  ###Human:\nWhen should a writ petition be file...   
2  ###Human:\nWhat is the procedure for filing a ...   
3  ###Human:\nWhat are the common reliefs sought ...   
4  ###Human:\nCan a plaint be amended after it ha...   

                                            question  \
0  What is the difference between a petition and ...   
1     When should a writ petition be filed in India?   
2  What is the procedure for filing a plaint in a...   
3  What are the common reliefs sought through a p...   
4  Can a plaint be amended after it has been file...   

                                              answer  
0  A petition is a formal request submitted to a ...  
1  A writ petition in India should be filed when ...  
2  To file a plaint in a civil case in Indiayou m...  
3  Public interest litigation (PIL) petitions in ...  
4  Yesa plaint can be amended in a civil case in ..

In [7]:
df2.head()

text  \
0  ###Human:\nWhat is the difference between a pe...   
1  ###Human:\nWhen should a writ petition be file...   
2  ###Human:\nWhat is the procedure for filing a ...   
3  ###Human:\nWhat are the common reliefs sought ...   
4  ###Human:\nCan a plaint be amended after it ha...   

                                            question  \
0  What is the difference between a petition and ...   
1     When should a writ petition be filed in India?   
2  What is the procedure for filing a plaint in a...   
3  What are the common reliefs sought through a p...   
4  Can a plaint be amended after it has been file...   

                                              answer  
0  A petition is a formal request submitted to a ...  
1  A writ petition in India should be filed when ...  
2  To file a plaint in a civil case in Indiayou m...  
3  Public interest litigation (PIL) petitions in ...  
4  Yesa plaint can be amended in a civil case in ...

In [8]:
df3 = pd.read_csv("hf://datasets/kshitij230/Indian-Law/Indian-Law.csv")

In [9]:
df3.rename(columns={'Instruction': 'question', 'Response': 'answer'}, inplace=True)
df3.to_csv('renamed_instruction_response_dataset.csv', index=False)

df3.head()

question  \
0  What is the difference between a petition and ...   
1     When should a writ petition be filed in India?   
2  What is the procedure for filing a plaint in a...   
3  What are the common reliefs sought through a p...   
4  Can a plaint be amended after it has been file...   

                                              answer  
0  A petition is a formal request submitted to a ...  
1  A writ petition in India should be filed when ...  
2  To file a plaint in a civil case in Indiayou m...  
3  Public interest litigation (PIL) petitions in ...  
4  Yesa plaint can be amended in a civil case in ...

In [10]:
from sklearn.model_selection import train_test_split 

merged_df = pd.concat([df1, df2, df3], ignore_index=True)

In [11]:
total_rows = len(merged_df)

In [12]:
train_test_df, validation_df = train_test_split(merged_df, test_size=500, random_state=42)
train_df, test_df = train_test_split(train_test_df, test_size=100, random_state=42)

In [13]:
train_df['split'] = 'train'
validation_df['split'] = 'validation'
test_df['split'] = 'test'

In [14]:
final_df = pd.concat([train_df, validation_df, test_df], ignore_index=True)

# Save the final dataset
file_path = 'Indian_legal_merged.csv'
final_df.to_csv(file_path, index=False)

file_path

'Indian_legal_merged.csv'